In [ ]:
%load_ext autoreload
%autoreload 2

import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
import aequitas
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot
from aequitas.preprocessing import preprocess_input_df
import ast

import matplotlib.pyplot as plt
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

dburl = os.environ['DBURL']
engine = create_engine(dburl)
pd.set_option('display.max_columns',100)
path_to_models = '/gpfs/data/dsapp-lab/triage-production_runs_small/trained_models/'
path_to_matrices = '/gpfs/data/dsapp-lab/triage-production_runs_small/matrices/'

-----

In [ ]:
best_mgs_by_for = [21140, 21145, 21111]
best_access_mg_id = 21144

In [ ]:
mgs_labels = {
    21144: 'Random Forest (1000 trees, max depth=2)',
        21140: "Random Forest (10 trees, no max depth)",
        21145: 'Random Forest (5000 trees, max depth=2)',
    21111: "Decision Tree (no max depth)",
    '21144.0': 'Random Forest (1000 trees, max depth=2)',
        '21140.0': "Random Forest (10 trees, no max depth)",
        '21145.0': 'Random Forest (5000 trees, max depth=2)',
    '21111.0': "Decision Tree (no max depth)",
     'jeff_baseline': 'Current System'
}

In [ ]:
model_group_to_use = 21111
def get_demographics():
    query = f"""with gender_race_info as (
        select distinct entity_id, 
            race_id, race,
            gender_id,
            case when transmission_category like '%%MSM%%' then 'MSM' else 'other' end as trans_categ_msm
        from features_cdph.demographics
        join features_cdph.trans_categ using (entity_id)
        join lookup_cdph.race using (race_id)
        )
        
        select model_id, p.entity_id, p.as_of_date,
                extract(year from p.as_of_date) as year,
                label_value, 
                race, gender_id as gender, trans_categ_msm
        from test_results.predictions p
        left join gender_race_info using (entity_id)
        join model_metadata.models using (model_id)
        where model_group_id = {model_group_to_use}
        """
    demo = pd.read_sql(query, engine)
    return(demo)
demo = get_demographics() # just demographics; combine with predictions later
demo.head()

In [ ]:
def get_models_same_mg(mg_id):
    q = f"""
        select model_id, train_end_time from model_metadata.models
        where model_group_id = {mg_id}
        """
    df = pd.read_sql(q, engine)
    return(df)

In [ ]:
def get_predictions(m_id, baseline=False):
    q = f"""
        select model_id, entity_id, as_of_date,
            score
        from test_results.predictions p
        where model_id ={m_id}
        """
    df = pd.read_sql(q, engine)
    if baseline:
        # replace score with new score
        q = f"""
            with vl as (
            select model_id, p.entity_id, as_of_date,
                date_col, vl
            from test_results.predictions p
            join features_cdph.viral_load vl
                on p.entity_id = vl.entity_id
                and vl.date_col < p.as_of_date
            where model_id ={m_id}
            order by entity_id, as_of_date, date_col
            )
            select distinct model_id, entity_id, as_of_date,
                last_value(vl) over (partition by entity_id, as_of_date) as latest_vl
            from vl
            order by as_of_date
        """
        df = pd.read_sql(q, engine)
        df['score'] = df['latest_vl'].rank(pct=True)
    return (df)



In [ ]:
def get_p_at_(m_id):
    if 'bs' in str(m_id):
        return({'value':None})
    q = f"""
        select value
        from test_results.evaluations e
        where metric='precision@' and parameter='1.0_pct'
        and model_id={m_id}
        """
    df = pd.read_sql(q, engine)
    return(df.loc[0].to_dict())

# Audition Waterfall Plot

In [ ]:
model_comment = 'CDPH_12months_access'
q = f'''
    select distinct model_group_id from model_metadata.models
    where model_comment = '{model_comment}'
    '''
model_groups = pd.read_sql(q, engine)

In [ ]:
access_models_for = pd.read_csv('bias_p1.csv')
access_models_for.tail()

In [ ]:
thresholds = {'rank_pct':[0.01]}

def get_for(m_id, pred):
    for_race = []
    # merge with demo
    df = pd.merge(pred, demo, on=['entity_id', 'as_of_date'])
    df = df.loc[df.groupby('entity_id')['label_value'].idxmax(), :]
    df = df[["entity_id", "score", "label_value", "race", "gender", "trans_categ_msm"]]
    g = Group()
    xtab, _ = g.get_crosstabs(df, thresholds)
    for_black = xtab.loc[xtab.attribute_value=='Black / African American', 'for'].values[0]
    for_white = xtab.loc[xtab.attribute_value=='White', 'for'].values[0]
    for_race = for_black/for_white
    
    for_male =  xtab.loc[xtab.attribute_value=='male', 'for'].values[0]
    for_female =  xtab.loc[xtab.attribute_value=='female', 'for'].values[0]
    for_trans_f =  xtab.loc[xtab.attribute_value=='transgender female', 'for'].values[0]
    
    for_1 = xtab.loc[xtab.attribute_value=='MSM', 'for'].values[0]
    for_2 = xtab.loc[xtab.attribute_value=='other', 'for'].values[0]
    for_msm_v_other = for_1/for_2
    return({'b_v_w':for_race, 
            'gender': [for_male, for_female, for_trans_f], 
            'trans_categ_msm': for_msm_v_other
           })

In [ ]:
# Looking at models with lower disparity in FOR
access_models_for = pd.DataFrame()
#demo = get_demographics() # just demographics; combine with predictions later



for mg_id in model_groups.model_group_id.values:
    print("----------------------------", mg_id)
    for_race = []
    for i, (m) in get_models_same_mg(mg_id).iterrows():
        m_id = m['model_id']
        pred = get_predictions(m_id)
        for_all = get_for(m_id, pred)
        p_at_10 = get_p_at_(m_id)
        access_models_for = access_models_for.append({'model_group_id': mg_id, 'model_id': m_id,
                            'train_end_time': m['train_end_time'],
                            'p_at_10': p_at_10['value'],
                            'for_b_v_w': for_all['b_v_w'],
                            'for_m_v_f': for_all['gender'][0]/for_all['gender'][1],
                            'for_trans_categ_msm': for_all['trans_categ_msm']
                       }, ignore_index=True)

In [ ]:
access_models_for['p_at_1']=access_models_for.model_id.apply(lambda mid: get_p_at_(mid)['value'])
access_models_for.head()

In [ ]:
# get baseline
print("Jeff baseline")
for i, (m) in get_models_same_mg(best_access_mg_id).iterrows():
    m_id = m['model_id']
    pred = get_predictions(m_id, baseline=True)
    for_all = get_for(m_id, pred)
    p_at_10 = get_p_at_(m_id)
    pred['rank_pct'] = pred.score.rank(pct=True)
    pred['outcome'] = pred.rank_pct.rank(pct=True)
    p = pd.merge(pred, demo, on=['entity_id', 'as_of_date'])
    top = p[p.rank_pct>0.99]
    p_at_1 = top.label_value.sum()/top.label_value.count()
    access_models_for = access_models_for.append({'model_group_id': 'jeff_baseline', 'model_id': 'bs'+str(m['model_id']),
                            'train_end_time': m['train_end_time'],
                            'p_at_10': p_at_10['value'],
                            'p_at_1': p_at_1,
                            'for_b_v_w': for_all['b_v_w'],
                            'for_m_v_f': for_all['gender'][0]/for_all['gender'][1],
                            'for_trans_categ_msm': for_all['trans_categ_msm']
                       }, ignore_index=True)

In [ ]:
#access_models_for.to_csv('bias_p1.csv', index=False)
access_models_for.tail()

In [ ]:
access_models_for = access_models_for.drop('distance', axis=1)

In [ ]:
def get_distance_from_best(disparity_col):
    # best = 1 = no disparity
    n_times = access_models_for.train_end_time.nunique()
    # is this the best way of computing distance?
    d_col = 'distance_'+disparity_col
    if d_col not in access_models_for.columns:
        access_models_for['distance_'+disparity_col] = abs(access_models_for[disparity_col]-1)

    # x = distance
    # y = number of times
    # each line is a model group
    df_dist_from_best = pd.DataFrame()
    for distance in np.arange(0,1,0.01):
        for mg_id, d in access_models_for.groupby('model_group_id'):
            df_dist_from_best = df_dist_from_best.append({'model_group_id': mg_id, 
                            'distance': distance,
                            'pct_of_time': d[d[d_col] < distance].shape[0]/n_times,
                       }, ignore_index=True)    
    return(df_dist_from_best)

In [ ]:
dist_from_best = {}

In [ ]:
for disparity_col in ['for_b_v_w', 'for_m_v_f', 'for_trans_categ_msm']:
    if disparity_col not in dist_from_best:
        print("calculating distance from best for "+disparity_col+" ...")
        dist_from_best[disparity_col] = get_distance_from_best(disparity_col)
    
    sns.set_style("whitegrid")
    fig, ax = plt.subplots(1,figsize=(24, 10))
    sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 2,"lines.markersize":12})

    for mg_id, d in dist_from_best[disparity_col].groupby('model_group_id'):
        if mg_id in ['21140.0', '21145.0', '21111.0', "jeff_baseline"]:
            _ = plt.plot(d.distance, d.pct_of_time, 
                 label=mg_id, marker='', linestyle='-', linewidth=10)
        else:
            _ = plt.plot(d.distance, d.pct_of_time, 
                 label=mg_id, marker='', linestyle='-')

    _ = plt.xlabel("Distance from 1 for "+ disparity_col, fontsize=24)
    _ = plt.ylabel("Percentage of time", fontsize=24)
    _ = plt.legend(ncol=4) #bbox_to_anchor=(0., .85, 1., .102), loc='upper center', ncol=2, borderaxespad=0., fontsize=24)
    plt.show()
#sns.despine()

In [ ]:
# they basically reach 1 at similar points
dist_from_best[disparity_col].groupby('model_group_id').apply(lambda d: d[d.pct_of_time<1].tail(1).distance)

---------

In [ ]:
metric = 'precision@'
parameter = '1.0_pct'
sns.set_style("whitegrid")
fig, ax = plt.subplots(1,figsize=(24, 10))
sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 4,"lines.markersize":12})
#_ = plt.ylim(0,1)
_ = plt.ylabel(f'{metric}:{parameter}')

for model_group_id in best_mgs_by_for + [best_access_mg_id, 'jeff_baseline']:
    d = access_models_for[access_models_for.model_group_id == str(model_group_id)+".0"].sort_values('train_end_time')
    _ = plt.plot(d.train_end_time, d.for_b_v_w, label=mgs_labels[model_group_id],
                 marker='o',linestyle='-', alpha=0.8)

_ = plt.ylabel(r"$\frac{FOR_{Black}}{FOR_{White}}$", fontsize=24)
_ = plt.xlabel("Year of Appointment for Validation Cohort", fontsize=24)
_ = plt.ylim(0.5, 1.8)    
_ = ax.axhspan(0.9, 1.1, alpha=0.3, color='#6A659999')
   
_ = plt.legend() #bbox_to_anchor=(0., .85, 1., .102), loc='upper center', ncol=2, borderaxespad=0., fontsize=24)
#sns.despine()

In [ ]:
mgs_to_plot = [str(x)+".0" for x in best_mgs_by_for+[best_access_mg_id]]+['jeff_baseline']
mgs_to_plot

In [ ]:
disp = access_models_for[access_models_for.model_group_id.isin(mgs_to_plot)]
disp.model_group_id.unique()
disp = disp.groupby('model_group_id')[['for_b_v_w', 'p_at_1']].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])
disp.columns = ['_'.join(col).strip() for col in disp.columns.values]
disp
disp.columns

In [ ]:
def plot_stuff(plt, d, label):
    #r = d.for_max - d.for_min
    _ = plt.plot(d.p_at_1_mean, d.for_b_v_w_mean, marker='o', linestyle='', label=label)
    c = _[0].get_color()
    _ = plt.errorbar(d.p_at_1_mean, d.for_b_v_w_mean, 
                 xerr=[d.p_at_1_mean-d['p_at_1_25%'],d['p_at_1_75%']-d.p_at_1_mean], 
                 marker='', linestyle='', color=c, linewidth=10)
    _ = plt.errorbar(d.p_at_1_mean, d.for_b_v_w_mean, 
                 xerr=[d.p_at_1_mean-d['p_at_1_5%'],d['p_at_1_95%']-d.p_at_1_mean], 
                 marker='', linestyle='', color=c)
    _ = plt.errorbar(d.p_at_1_mean, d.for_b_v_w_mean, 
                 yerr=[d.for_b_v_w_mean-d['for_b_v_w_25%'],d['for_b_v_w_75%']-d.for_b_v_w_mean], 
                 marker='', linestyle='', color=c, linewidth=10)
    _ = plt.errorbar(d.p_at_1_mean, d.for_b_v_w_mean, 
                 yerr=[d.for_b_v_w_mean-d['for_b_v_w_5%'],d['for_b_v_w_95%']-d.for_b_v_w_mean], 
                 marker='', linestyle='', color=c)
    return(plt)

sns.set_style('whitegrid')                                                                                                                                      
sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 2.25,"lines.markersize":20})                                                                  
fig, ax = plt.subplots(1,1,figsize=(24,10))

for i, row in disp.groupby('model_group_id'):
    plt = plot_stuff(plt, row, mgs_labels[i])
_ = ax.axhspan(0.9, 1.1, alpha=0.3, color='#6A659999')
_ = plt.ylabel(r'$\frac{FOR_{Black}}{FOR_{White}}$', fontsize=40)
_ = plt.xlabel("Average Positive Predictive Value for top 1%")
_ = plt.ylim(0.5, 1.8)
h, l = ax.get_legend_handles_labels()
new_h = []
new_l = []
for i in range(0, len(h)):
    if 'mean' in l[i]:
        continue
    new_h.append(h[i])
    new_l.append(l[i])
_ = plt.legend(new_h, new_l, ncol=2, loc="upper center")
plt.show()
